# 03 Pytorch Dataset
#### ＝＝＝ 目次 ＝＝＝
0. ライブラリの呼び出し
1. transforms
2. Dataset
3. DataLoader
4. 学習・予測

補足資料

5. tqdmを用いたプログレスバーへの学習結果の出力

#### PyTorchで自作データセットを利用するために以下を定義する
- transforms：データの前処理を記述するモジュール
- Dataset：データをtransformsしてラベルと合わせて返すモジュール
- DataLoader：学習のためにデータをバッチサイズに分割してイテレータを返す

---
## 0. ライブラリの呼び出し

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm

---
## 使用するデータセット：MNIST
- 0～9の10クラスの手書き数字画像
- サイズ：28×28のグレースケール(チャンネル数1)

In [ ]:
image_path = './dataset/train/0/0000.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
print(image.shape)
plt.imshow(image, cmap="gray")
plt.show()

---
## 1. transforms
- [torchvision.transforms](https://pytorch.org/docs/stable/torchvision/transforms.html)：画像データに対する前処理を記述するモジュール
- 正規化や標準化，Data Augmentation(DA)など
- 複数の前処理を行う場合は`transforms.Compose`を用いる

|<div align='center'>前処理</div>|<div align='center'>意味</div>|
|---|---|
|<div align='left'>Compose(transforms)</div>|<div align='left'>複数の前処理をリストとして受け取り結合する</div>|
|<div align='left'>ToTensor()</div>|<div align='left'>PIL Image or numpy.ndarray(H, W, C) range [0, 255] → tensor(C, H, W) range [0, 1] (正規化)</div>|
|<div align='left'>Normalize(mean, std)</div>|<div align='left'>指定した値で正規化 (input - mean) / std</div>|
|<div align='left'>Resize(size)</div>|<div align='left'>指定したサイズにリサイズ</div>|
|<div align='left'>CenterCrop(size)</div>|<div align='left'>画像中央をsize×sizeで切り取り</div>|
|<div align='left'>RandomResizedCrop(size, scale, ratio)</div>|<div align='left'>(DA)指定した比率のサイズとアスペクト比でトリミング</div>|
|<div align='left'>RandomHorizontalFlip(p)</div>|<div align='left'>(DA)指定した確率で水平反転</div>|
|<div align='left'>Lambda(lambd)</div>|<div align='left'>自作lambdaを適用</div>|

`transforms.ToTensor()`

In [ ]:
transform = transforms.ToTensor()
transformed_image = transform(image)

print(type(transformed_image))
print(transformed_image.shape)
print(transformed_image.min(), transformed_image.max())

`transforms.Compose(transforms)`

In [ ]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5,), std=(1,))])
transformed_image = transform(image)

print(type(transformed_image))
print(transformed_image.shape)
print(transformed_image.min(), transformed_image.max())

## transformsをクラスとして定義
train, validation, testそれぞれで前処理を分ける場合(特にDAなど)，クラスとして定義すると便利                      

In [ ]:
class ImageTransform():
    
    def __init__(self):
        self.data_transform = {
            'train': transforms.Compose([transforms.ToTensor()]),
            'validation': transforms.Compose([transforms.ToTensor()]),
            'test': transforms.Compose([transforms.ToTensor()])
        }

    def __call__(self, image, phase='train'):
        return self.data_transform[phase](image)

In [ ]:
image_path = './dataset/train/0/0000.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

transform = ImageTransform()
transformed_image = transform(image, "train")

print(type(transformed_image))
print(transformed_image.shape)
print(transformed_image.min(), transformed_image.max())

---
## 2. Dataset
`torch.utils.data.Dataset`：データをtransformsしてラベルと合わせて返すモジュール

`Dataset`を継承して`__init__`，`__len__`，`__getitem__`メソッドを定義したクラスを作成
- `__init__`：コンストラクタ(初期化メソッド)
- `__len__`：画像の枚数を返す
- `__getitem__`：前処理をした画像のTensor形式のデータとラベルを返す

In [ ]:
# MNISTのDatasetを作成する

class MNISTDataset(data.Dataset):

    def __init__(self, phase='train', transform=None):
        
        target_path = os.path.join('dataset', phase, '**/*.png')
        path_list = glob(target_path)
        
        images = []
        labels = []
        
        for path in tqdm(path_list):
            image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            label = int(path.split(os.sep)[2]) # 画像のラベルをファイル名から取得
            images.append(image)
            labels.append(label)
        
        self.transform = transform
        self.phase = phase
        self.images = images
        self.labels = labels
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):

        # index番目の画像，ラベル
        image = self.images[index]  # H×W×C
        label = self.labels[index]

        image_transformed = self.transform(image, self.phase) # C×H×W

        return image_transformed, label

In [ ]:
# 実行
train_dataset = MNISTDataset(phase='train', transform=ImageTransform())
val_dataset = MNISTDataset(phase='validation', transform=ImageTransform())

In [ ]:
# 動作確認
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

---
## 3. DataLoader
`torch.utils.data.DataLoader(dataset, batch_size, shuffle)`：データをバッチサイズに分割してイテレータ$^{*1}$を返す関数
- `dataset`：2で作成した`Dataset`
- `batch_size`：ミニバッチのサイズ
- `shuffle`：dataの参照の仕方をランダムにするか否か

$^{*1}$ 要素を反復して取り出すことのできるオブジェクト(list, tuple, rangeなどの総称)

In [ ]:
batch_size = 32

# DataLoaderを作成
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "validation": val_dataloader}

In [ ]:
# 動作確認
inputs, labels = list(dataloaders_dict["train"])[0] # 0番目のバッチ
print(inputs.shape)
print(labels)

---
## 4. 学習
0. deviceを定義(GPU or CPU)
1. 学習データを用意
2. モデルの定義
3. loss関数，optimizerを定義
4. データをモデルに入力しlossを計算し，backpropagation，パラメータ更新

#### 0. deviceを定義(GPU or CPU)

In [ ]:
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

#### 1. 学習データを用意

In [ ]:
batch_size = 32

# Datasetを作成
train_dataset = MNISTDataset(phase='train', transform=ImageTransform())
val_dataset = MNISTDataset(phase='validation', transform=ImageTransform())

# DataLoaderを作成
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "validation": val_dataloader}

#### 2. モデルの定義

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
                                  nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2))
        
        self.fc = nn.Sequential(nn.Linear(64 * 7 * 7, 120), nn.ReLU(inplace=True),
                                nn.Linear(120, 84), nn.ReLU(inplace=True),
                                nn.Linear(84, 10))

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x.view(-1, 64 * 7 * 7))
        return x

In [ ]:
# モデルの定義
model = Net()

# マルチGPUが使える場合
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model.to(device) # モデルをGPUへ

#### 3. loss関数，optimizerを定義

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

#### 4. データをモデルに入力しlossを計算し，backpropagation，パラメータ更新

In [ ]:
epochs = 2
history = {"train_loss":[], "val_loss":[], "train_acc":[], "val_acc":[]} # 学習曲線用

# ネットワークがある程度固定であれば、高速化させる
torch.backends.cudnn.benchmark = True

for epoch in range(epochs):
    print('Epoch：{}/{}'.format(epoch+1, epochs))
    
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
            
        epoch_loss = 0.0
        epoch_corrects = 0
        
        for inputs, labels in tqdm(dataloaders_dict[phase]):
            
            # GPUが使えるならGPUにデータを送る
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 順伝播
            with torch.set_grad_enabled(phase == 'train'):
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)  # ラベルを予測

                # train時は学習
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                epoch_loss += loss.item() * inputs.size(0)        # lossの合計を更新
                epoch_corrects += torch.sum(preds == labels.data) # 正解数の合計を更新

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
        
        # historyにlossとaccuracyを保存
        if phase == "train":
            history["train_loss"].append(epoch_loss)
            history["train_acc"].append(epoch_acc)
        else:
            history["val_loss"].append(epoch_loss)
            history["val_acc"].append(epoch_acc)

        print('{} loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

#### モデルを保存

In [ ]:
#保存用ディレクトリを作成
import os
os.makedirs("result", exist_ok=True)

# モデルを保存する
torch.save(model.to('cpu').state_dict(), "result/model_MNIST.pth")

#### 学習曲線

In [ ]:
plt.title("loss")
plt.plot(history["train_loss"], label="train")
plt.plot(history["val_loss"], label="validation")
plt.legend(loc="best")

In [ ]:
plt.title("accuracy")
plt.plot(history["train_acc"], label="train")
plt.plot(history["val_acc"], label="validation")
plt.legend(loc="best")

---
## 予測

In [ ]:
batch_size = 32

# Datasetを作成
test_dataset = MNISTDataset(phase='test', transform=ImageTransform())

# DataLoaderを作成
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 保存したモデルを読み込む。
model = Net()
model.load_state_dict(torch.load("result/model_MNIST.pth"))

# マルチGPUが使える場合
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model.to(device) # モデルをGPUへ

In [ ]:
correct = 0

with torch.no_grad():
    for inputs, labels in tqdm(test_dataloader):
        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)
test_acc = correct.double() / len(test_dataloader.dataset)
print("Test Accuracy :", test_acc.item())

---
# 補足資料
## 5. tqdmを用いたプログレスバーへの学習結果の出力
`tqdm(iterable, desc, postfix)`：処理の進捗状況をプログレスバーとして表示するためのパッケージ
- iterable：イテレーター
- desc：プログレスバーの前に表示する情報
- postfix：プログレスバーの前に表示する情報

In [ ]:
from time import sleep

for n in tqdm(range(100), desc="[train]", postfix={"loss":0.5, "acc":0.6}):
    sleep(0.01)

ループの中で動的に変更させたい場合は以下のようにする．

In [ ]:
with tqdm(range(100)) as pbar:
    for i, n in enumerate(pbar):
        
        # プログレスバーの前に表示
        pbar.set_description("Epoch {} [train] ".format(i))
        
        # プログレスバーの後ろに表示
        pbar.set_postfix({"loss":i, "acc":i})
        sleep(0.01)